In [242]:
%load_ext autoreload

%autoreload 2
from align import *
from typing import Tuple
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [261]:
def _get_next_cell(self, i,j):
    
    """Returns coordinates of next cell to go to in traceback according to move encoded in traceback matrix"""
    
    move = self._traceback_matrix[i,j]
    
    #move diagonally
    if move == 0:
        return (i-1, j-1)
    #move left
    if move == 1:
        return (i, j-1)
    #move up
    if move == 2:
        return (i-1, j)

In [262]:
seqA, _ = read_fasta("./data/test_seq3.fa")
seqB, _ = read_fasta("./data/test_seq4.fa")

In [263]:
t = NeedlemanWunsch(sub_matrix_file='substitution_matrices/BLOSUM62.mat', gap_open=-10, gap_extend=-1)
sub_matrix = t.sub_dict
sub_dict = t.sub_dict

gap_open = -10
gap_extend = -1

In [264]:
#initialize matrices
_matrix_dims = (len(self._seqA)+1, len(self._seqB)+1)
self._align_matrix = np.full(_matrix_dims, -np.inf)
self._gapA_matrix = np.full(_matrix_dims, -np.inf)
self._gapB_matrix = np.full(_matrix_dims, -np.inf)
self._traceback_matrix = np.full(_matrix_dims, -np.inf)

self._align_matrix[0,0] = 0

self._traceback_matrix[1:,0] = 1
self._traceback_matrix[0,1:] = 2 
self._traceback_matrix[0,0] = 0

for i in range(_matrix_dims[0]): 
    if i == 0:
        self._gapA_matrix[i, 0] = self.gap_open
    else:
        self._gapA_matrix[i, 0] = self.gap_extend+self._gapA_matrix[i-1,0]
    
for j in range(_matrix_dims[1]):   
    if j == 0:
        self._gapB_matrix[0, j] = self.gap_open
    else:
        self._gapB_matrix[0, j] = self.gap_extend+self._gapB_matrix[0,j-1]
    
#iterate by row and column

for i in range(1, _matrix_dims[0]):
    for j in range(1, _matrix_dims[1]):
        
        subst_score = sub_matrix[(self._seqA[i-1], self._seqB[j-1])]
        options = [subst_score + self._align_matrix[i-1, j-1]]
        options += [subst_score + self._gapA_matrix[i-1,j-1]]
        options += [subst_score + self._gapB_matrix[i-1,j-1]]
        self._align_matrix[i,j] = max(options)
        
        self._gapA_matrix[i, j] = max(self._align_matrix[i-1,j]+self.gap_open+self.gap_extend, 
                                self._gapA_matrix[i-1, j]+self.gap_extend)
        self._gapB_matrix[i, j] = max(self._align_matrix[i,j-1]+self.gap_open+self.gap_extend, 
                                self._gapB_matrix[i, j-1]+self.gap_extend)
        options = [self._align_matrix[i,j],self._gapB_matrix[i, j],self._gapA_matrix[i, j],]
        
        #add best move to traceback matrix
        self._traceback_matrix[i,j] = options.index(max(options))
        
#traceback
self.seqA_align = ''
self.seqB_align = ''

_seqA_pos = _matrix_dims[0]-1
_seqB_pos = _matrix_dims[1]-1

self.alignment_score = self._align_matrix[_seqA_pos, _seqB_pos]

while True:

    move = self._traceback_matrix[_seqA_pos, _seqB_pos]
    _seqA_prev_pos = _seqA_pos
    _seqB_prev_pos = _seqB_pos

    _seqA_pos, _seqB_pos = get_next_cell(_seqA_prev_pos, _seqB_prev_pos)

    if _seqA_pos == _seqA_prev_pos:
        self.seqA_align += '-'
    else:
        self.seqA_align += self._seqA[_seqA_pos]
    
    if _seqB_pos == _seqB_prev_pos:
        self.seqB_align += '-'
    else:
        self.seqB_align += self._seqB[_seqB_pos]
        
    if _seqA_pos == 0 and _seqB_pos == 0:
        break

self.seqA_align = self.seqA_align[::-1]
self.seqB_align = self.seqB_align[::-1]

4 3
3 2
2 1
1 1
MYQR
M-QR
